In [12]:
from langchain_openai import OpenAI
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.chains import RetrievalQA



In [13]:
%pip install --quiet langchain-openai
%pip install --quiet langchain-community


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [17]:
loader= TextLoader("/content/Metallica.txt")

In [18]:
documents= loader.load()

In [19]:
documents

[Document(metadata={'source': '/content/Metallica.txt'}, page_content='Metallica\xa0is an American\xa0heavy metal\xa0band. It was formed in\xa0Los Angeles\xa0in 1981 by vocalist and guitarist\xa0James Hetfield\xa0and drummer\xa0Lars Ulrich, and has been based in\xa0San Francisco\xa0for most of its career.[1][2]\xa0The band\'s fast tempos, instrumentals and aggressive musicianship made them one of the founding "big four" bands of\xa0thrash metal, alongside\xa0Megadeth,\xa0Anthrax\xa0and\xa0Slayer. Metallica\'s current lineup comprises founding members and primary songwriters Hetfield and Ulrich, longtime lead guitarist\xa0Kirk Hammett\xa0and bassist\xa0Robert Trujillo. Guitarist\xa0Dave Mustaine, who formed Megadeth after being fired from Metallica, and bassists\xa0Ron McGovney,\xa0Cliff Burton\xa0and\xa0Jason Newsted\xa0are former members of the band. The band is one of the\xa0most commercially successful bands of all time, having sold more than 163\xa0million albums worldwide as of 20

In [20]:
with open("/content/Metallica.txt", "r", encoding="utf-8") as file:
    text = file.read()


In [21]:
word_count=len(text.split())

In [22]:
print(word_count)

526


In [23]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len,
)


In [24]:
docs= text_splitter.split_documents(documents)

In [25]:
docs[0]

Document(metadata={'source': '/content/Metallica.txt'}, page_content='Metallica\xa0is an American\xa0heavy metal\xa0band. It was')

In [26]:
len(docs)

85

In [27]:
embeddings= OpenAIEmbeddings()

In [31]:
pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 31.9 MB/s eta 0:00:00


In [32]:
import faiss
print(faiss.__version__)


1.11.0


In [33]:
from langchain_community.vectorstores import FAISS
library= FAISS.from_documents(docs, embeddings)

In [34]:
Query1 = "Who is the lead singer of Metallica?"
Query_answer = library.similarity_search(Query1)



In [35]:
print(Query_answer[0].page_content)

vocalist and guitarist James Hetfield and


In [36]:
print(Query_answer[1].page_content)

Metallica's current lineup comprises founding


In [37]:
docs_and_scores= library.similarity_search_with_score(Query1)

In [38]:
print(docs_and_scores[0])

(Document(id='9a332af6-7468-4829-88ed-e04ab2438573', metadata={'source': '/content/Metallica.txt'}, page_content='vocalist and guitarist\xa0James Hetfield\xa0and'), np.float32(0.3101305))


In [39]:
docs_and_scores[1]

(Document(id='aeb17c47-3f3d-4432-ac76-44ce9920aa98', metadata={'source': '/content/Metallica.txt'}, page_content="Metallica's current lineup comprises founding"),
 np.float32(0.31784117))

In [40]:
retriever= library.as_retriever()

In [43]:
qa= RetrievalQA.from_chain_type(llm=OpenAI(),chain_type="stuff", retriever= retriever)

In [44]:
retriever_query="What is the most hated Metallica Album"

In [45]:
results= qa.invoke(retriever_query)

In [46]:
print(results)

{'query': 'What is the most hated Metallica Album', 'result': " I don't know."}


In [49]:
retriever_query1="How many Studio albums have metallica released"

In [50]:
results1=qa.invoke(retriever_query)

In [51]:
results

{'query': 'How many Studio albums have metallica released',
 'result': ' Eleven'}

In [55]:
library.save_local("faiss_index_metallica")

In [60]:
metallica_saved= FAISS.load_local("faiss_index_metallica",embeddings, allow_dangerous_deserialization=True)

In [59]:
qa= RetrievalQA.from_chain_type(llm=OpenAI(),chain_type="stuff",retriever=metallica_saved.as_retriever())

In [61]:
results= qa.invoke(retriever_query)

In [62]:
print(results)

{'query': 'How many Studio albums have metallica released', 'result': ' 11'}


In [65]:
retriever_query2="Where was Metallica founded"
r2= qa.invoke(retriever_query2)
print(r2)

{'query': 'Where was Metallica founded', 'result': ' Metallica was founded in Los Angeles, California.'}
